In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import Constants

In [2]:
box_scores = os.listdir(Constants.SCORE_DIR)

In [4]:
box_scores = [os.path.join(Constants.SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [6]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    
    return soup

In [9]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "Team"
    cols[-1] = "Total"
    line_score.columns = cols
    
    line_score = line_score[["Team", "Total"]]
    return line_score

In [11]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [31]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    
    return season

In [38]:
base_cols = None
games = []

for box_score in  box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["Team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
            
        summary = summary[base_cols]
        
        summaries.append(summary)
        
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["Total"] > full_game["Total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")


/var/folders/t2/p3yp6v4544jds05t77dbbtq40000gp/T/ipykernel_21397/1869830524.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
/var/folders/t2/p3yp6v4544jds05t77dbbtq40000gp/T/ipykernel_21397/385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
/var/folders/t2/p3yp6v4544jds05t77dbbtq40000gp/T/ipykernel_21397/385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
/va

In [39]:
games_df = pd.concat(games, ignore_index=True)

In [41]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,Team_opp,Total_opp,home_opp,season,date,won
0,240.0,240.0,42.0,92.0,0.457,17.0,41.0,0.415,23.0,29.0,...,25.0,34.5,160.0,124.0,POR,121,1,2022,2021-10-20,True
1,240.0,240.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,...,16.7,36.4,175.0,120.0,SAC,124,0,2022,2021-10-20,False
2,240.0,240.0,41.0,88.0,0.466,12.0,38.0,0.316,20.0,24.0,...,40.0,32.1,172.0,129.0,PHI,109,1,2022,2021-10-22,True
3,240.0,240.0,43.0,91.0,0.473,14.0,36.0,0.389,9.0,11.0,...,21.2,31.1,213.0,118.0,BRK,114,0,2022,2021-10-22,False
4,240.0,240.0,44.0,88.0,0.500,24.0,54.0,0.444,9.0,17.0,...,31.8,31.4,163.0,130.0,ORL,96,1,2022,2021-10-22,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,240.0,240.0,36.0,87.0,0.414,14.0,37.0,0.378,12.0,17.0,...,40.0,33.2,158.0,109.0,DEN,110,0,2022,2021-10-20,False
68,240.0,240.0,48.0,91.0,0.527,13.0,32.0,0.406,14.0,17.0,...,50.0,46.4,205.0,128.0,CLE,112,1,2022,2021-10-22,True
69,240.0,240.0,43.0,84.0,0.512,9.0,31.0,0.290,17.0,20.0,...,100.0,70.6,173.0,117.0,CHO,123,0,2022,2021-10-22,False
70,240.0,240.0,38.0,99.0,0.384,10.0,34.0,0.294,9.0,15.0,...,23.3,29.5,146.0,110.0,CLE,101,1,2022,2021-10-23,False


In [ ]:
games_df.to_csv("nba_games.csv")